# Import libraries

In [1]:
# Import libraries
import os
import pandas as pd
import json
import sacrebleu

# Set Paths

In [2]:
# Base path
base_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))

# Model Parameters
unsloth_model_name = 'unsloth/gemma-2-2b-it-bnb-4bit'
company_name = 'alphabet'
model_name = unsloth_model_name.split('/')[1]

# Results path
results_path_file = os.path.join(base_path, 'results', company_name, model_name, 'merged_results.json')

# Print paths
print('Base path:', base_path)
print('Results path:', results_path_file)

Base path: /cs/student/msc/csml/2023/ngriessh/historical_mt
Results path: /cs/student/msc/csml/2023/ngriessh/historical_mt/results/alphabet/gemma-2-2b-it-bnb-4bit/merged_results.json


# Set Evaluation Parameters

In [25]:
# Choose source language
source_language = "Early Modern Bohemian German"
target_language = "English" if source_language == "Early Modern Bohemian German" else "Early Modern Bohemian German"

# Evaluate finetuning and icl?
evaluate_icl = False
evaluate_finetuning = True

# Median splitting with regard to word count?
median_splitting = False
# Translation direction is defined depending on the chosen source languag 
translation_direction = "DE_to_EN" if source_language == "Early Modern Bohemian German" else "EN_to_DE"

# Print
print(f'Source Language: {source_language}')
print(f'Target Language: {target_language}')
print(f'Translation Direction: {translation_direction}')
print(f'Evaluate ICL: {evaluate_icl}')
print(f'Evaluate finetuning: {evaluate_finetuning}')
print(f'Median Splitting? {median_splitting}')

Source Language: Early Modern Bohemian German
Target Language: English
Translation Direction: DE_to_EN
Evaluate ICL: False
Evaluate finetuning: True
Median Splitting? False


# Load Results File

In [26]:
# Load the JSON file
with open(results_path_file, 'r') as f:
    data = [json.loads(line) for line in f]

# Create a DataFrame based on the JSON file
merged_df = pd.DataFrame(data)

# Check Columns

In [27]:
# Check column names
merged_df.columns

Index(['Early Modern Bohemian German', 'English', 'DE_to_EN_finetuning',
       'EN_to_DE_finetuning'],
      dtype='object')

# Construct Candidate Columns

In [28]:
# Define a function to generate candidate columns based on the conditions
def construct_candidate_columns(translation_direction, evaluate_icl, evaluate_finetuning):
    shots = ['000', '001', '002', '004'] #, '008', '016', '032', '064', '128']
    
    # Add shots columns
    if evaluate_icl:
        if translation_direction == 'DE_to_EN':
            candidate_columns = [f'DE_to_EN_{shot}_example_prompt' for shot in shots]
        elif translation_direction == 'EN_to_DE':
            candidate_columns = [f'EN_to_DE_{shot}_example_prompt' for shot in shots]
    
    # Add finetuning column
    if evaluate_finetuning:
        if evaluate_icl:
            candidate_columns.append(f'{translation_direction}_finetuning')
        else:
            candidate_columns = [f'{translation_direction}_finetuning']
    
    return candidate_columns

# You can now concatenate or use these columns as needed
candidate_columns = construct_candidate_columns(translation_direction, evaluate_icl, evaluate_finetuning)
candidate_columns

['DE_to_EN_finetuning']

# Median Splitting

In [29]:
# Compute word count for the source_language column
source_language_word_count = f'{source_language}_word_count'
merged_df[source_language_word_count] = merged_df[source_language].apply(lambda x: len(str(x).split()))

# Compute the median word count
median_word_count = merged_df[source_language_word_count].median()

# Create merged_df_short and merged_df_long based on the median
merged_df_short = merged_df[merged_df[source_language_word_count] <= median_word_count]
merged_df_long = merged_df[merged_df[source_language_word_count] > median_word_count]

#vPrint the shapes of the datasets
print("Shape of merged_df_short:", merged_df_short.shape)
print("Shape of merged_df_long:", merged_df_long.shape)
print(f'Median word count in {source_language}: {median_word_count}')

Shape of merged_df_short: (501, 5)
Shape of merged_df_long: (499, 5)
Median word count in Early Modern Bohemian German: 73.0


In [30]:
merged_df_short.head()

,Early Modern Bohemian German,English,DE_to_EN_finetuning,EN_to_DE_finetuning,Early Modern Bohemian German_word_count
0,1. Ein Ambt befehlich an h. Joachim vom Eberha...,1. An manorial court command is to be made to ...,A manorial court's command was issued on appli...,An einen Ambt befehlich an den h.: Jochim Vonn...,45
4,Ao 1661. Martius. Mildenaw. Christoph blumbrig...,1661. march. Mildenaw. Christoph Blumbrig's co...,March Anno 1661 in mildenau; complaint of Chri...,Anno p 1661 Martij,16
5,"Schultes, Vnd Schoppen Zue Bernsdorff haben si...",16th July. Schulthess and jurymen in Bernsdorf...,village headman and jurymen in Bernsdorf have ...,Den 16 Julij Scholtess vnd Schöppen Zue Bernßd...,12
6,1782. Ist der MildenEicher Scholtes mit dem Di...,1782. The Mildeneichen village headman stood f...,On 30 Sep there was a public manorial court's ...,Der Mieldenaichsche scholtze hat sich bej dem ...,23
8,3. Novembris. Matz Apelt beschweret sich Vber ...,3. Novembris. Matz Apelt complains against Jac...,On 3 November; Matz Apel complains against Jac...,Den 3 Nouembristes Claget matz Apeltt Vber Jak...,68


In [31]:
merged_df_long.head()

,Early Modern Bohemian German,English,DE_to_EN_finetuning,EN_to_DE_finetuning,Early Modern Bohemian German_word_count
1,"1. die lehenß Vnderthanen im Winckel, sollen h...",1. The fief serfs in the Winckel shall hencefo...,The fief-serfs are supposed henceforth for ent...,1. die Lehen Vnterthanen im winckeln sollen hi...,109
2,1. dz Sie verschienen freytag als den 29 huig ...,"1. last Friday, the 29th, they came up to the ...",That on last Friday namely the 29th they came ...,1st Vnterschreibungen am Freytag Verwichener Z...,419
3,Es ist Vor.12. iharen Vnnd mehr Zu Oberweigßdo...,"12 years ago and more, in Oberweigsdorf, Paul ...",Twelve years ago in Upper Weigsdorff died off ...,Vor Vngefehr Zwantzig Jahren vnd mehr ist Zu o...,95
7,2. Ist errinnert Worden alle Undt iede alte re...,2. They were reminded to pay all arrears quick...,The reminder was made that all and every old R...,2. Ist ihnen erinnert worden alle Restiren bal...,183
10,Vor Vngefehr 14: tagen Kombt des Scholzens Zu ...,About 14 days ago the Scholz's in Bernßdorff's...,About fourteen days ago came before us here at...,Vor Vngefehr Achtage ist des scholzens Zu bern...,360


# BLEU Evaluation

In [32]:
# Check if median splitting is true
if median_splitting:
    dataset_splits = [merged_df_short, merged_df_long]
    dataset_names = ['merged_df_short', 'merged_df_long']
else:
    dataset_splits = [merged_df]
    dataset_names = ['merged_df']

# Compute BLEU scores for both datasets
for split, name in zip(dataset_splits, dataset_names):
    # Print statement
    print('______________________________________________________________________________________________')
    print(f'Current Dataset: {name}. Source language is {source_language}.')
    print('______________________________________________________________________________________________')

    # Create references for split
    references = [split[target_language].tolist()]

    # Compute corpus-level BLEU scores
    for col in candidate_columns:
        candidates = split[col].tolist()
        print(f'Evaluation of target language {target_language} and LLM-generated candidates {col}')
        bleu_score = sacrebleu.corpus_bleu(candidates, references).score
        print(f"Corpus-level SacreBLEU score for {col}: {bleu_score:.3f}\n")

______________________________________________________________________________________________
Current Dataset: merged_df. Source language is Early Modern Bohemian German.
______________________________________________________________________________________________
Evaluation of target language English and LLM-generated candidates DE_to_EN_finetuning
Corpus-level SacreBLEU score for DE_to_EN_finetuning: 6.409



# Create Text File to Inspect Translation Results

In [15]:
# Function to generate the inspection text for a specific entry
def generate_inspection_text(df, entry_index, translation_directions, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:

        # Get the row corresponding to the entry index
        row = df.iloc[entry_index]

        # Write the German and English translations pairs by Sheilagh Ogilvie
        f.write(f"Entry {entry_index + 1}:\n")
        f.write(f"Early Modern Bohemian German (Sheilagh Ogilvie's Transcription): {row['Early Modern Bohemian German']}\n")
        f.write(f"English (Sheilagh Ogilvie): {row['English']}\n\n")

        # Write the candidate translations for each translation direction
        for col in df.columns:
            if any(direction in col for direction in translation_directions):
                f.write(f"{col}: {row[col]}\n")
        
        f.write("\n" + "="*50 + "\n\n")

# Define the translation directions and the entry to inspect
translation_directions = ['DE_to_EN', 'EN_to_DE']
entry_index = 0

# Call the function to generate the text file
output_file = 'translation_inspection_entry.txt'
generate_inspection_text(merged_df, entry_index, translation_directions, output_file)
print(f"Text file created: {output_file}")


Text file created: translation_inspection_entry.txt


In [16]:
import matplotlib.pyplot as plt

# Function to compute word count and plot the histogram for any column
def plot_word_count_distribution(df, column_name):
    # Compute word count for each entry in the specified column
    df['word_count'] = df[column_name].apply(lambda x: len(x.split()))
    
    # Plotting the histogram of word count distribution
    plt.hist(df['word_count'], bins=10, edgecolor='black')
    plt.title(f'Word Count Distribution for {column_name}')
    plt.xlabel('Word Count')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

# Example usage with 'DE_to_EN_128_example_prompt'
plot_word_count_distribution(merged_df, 'DE_to_EN_004_example_prompt')

KeyError: 'DE_to_EN_004_example_prompt'